This notebook automates the creation of a daily telemetry report for the Western Formula Racing electric vehicle, pulling data directly from InfluxDB 3. It demonstrates a reproducible pipeline that queries, processes, and visualizes time-series sensor data for performance monitoring.

* Automatic date-based querying with local-to-UTC time conversion

* Deterministic randomness added to numeric data for open-source reproducibility (salted noise injection)

* Graceful handling of no-data days, generating placeholder PDFs when needed

* Matplotlib-based multi-panel PDF report showing voltage, current, and braking traces

* Integrated summary statistics formatted using tabulate for clean Markdown-style presentation

This project highlights data engineering automation, reproducible analytics, and clear visual reporting — a practical example of Python-driven telemetry analysis for high-frequency EV datasets.

In [1]:
from influxdb_client_3 import InfluxDBClient3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime, timedelta
from tabulate import tabulate
from zoneinfo import ZoneInfo
import os

# Connect to InfluxDB 3
client = InfluxDBClient3(
    host=os.getenv("INFLUX_SERVER"),
    token=os.getenv("INFLUX_TOKEN"),
    database="WFR25"
)

In [2]:
# Check InfluxDB connection
try:
    # Simple query to test connection
    test_query = client.query("SELECT 1 as test")
    print("Connection to InfluxDB successful!")
    print("Test query result:", test_query)
except Exception as e:
    print(f"Connection failed: {e}")

Connection to InfluxDB successful!
Test query result: pyarrow.Table
test: int64 not null
----
test: [[1]]


Add deterministic randomness for public notebooks

In [3]:
import numpy as np, hashlib, os

# Salt seed based on date or Papermill parameter
salt_input = os.getenv("REPORT_DATE", "2025-06-20") + os.getenv("GITHUB_RUN_ID", "local")
salt = int(hashlib.sha256(salt_input.encode()).hexdigest(), 16) % (10**8)
np.random.seed(salt)
print(f"Salt seed: {salt}")

def salt_dataframe(df, scale_factor=0.05):
    df_salted = df.copy()
    for col in df_salted.columns:
        if np.issubdtype(df_salted[col].dtype, np.number):
            std = df_salted[col].std() or 1.0
            noise = np.random.normal(0, std * scale_factor, len(df_salted))
            df_salted[col] = df_salted[col] + noise
    return df_salted

Salt seed: 62722650


In [4]:
# Set the report date (Toronto local time)
# REPORT_DATE = "2025-06-19"  # <-- change or pass dynamically
REPORT_DATE = (datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")
DATABASE = "WFR25"

# Compute local → UTC time range
toronto_tz = ZoneInfo("America/Toronto")
start_local = datetime.strptime(REPORT_DATE, "%Y-%m-%d").replace(tzinfo=toronto_tz)
end_local = start_local + timedelta(days=1)

start_utc = start_local.astimezone(ZoneInfo("UTC"))
end_utc = end_local.astimezone(ZoneInfo("UTC"))

print(f"Report range (Toronto local): {start_local} → {end_local}")
print(f"Converted to UTC: {start_utc} → {end_utc}")

# Output path
os.makedirs("./output", exist_ok=True)
OUTPUT_PATH = f"./output/Run_Report_{REPORT_DATE}.pdf"

# Query
query_template = f"""
SELECT "time", "sensorReading", "signalName"
FROM "iox"."{DATABASE}"
WHERE "time" >= TIMESTAMP '{start_utc.isoformat()}'
  AND "time" < TIMESTAMP '{end_utc.isoformat()}'
  AND "signalName" IN (
    'INV_DC_Bus_Voltage',
    'INV_Phase_A_Current',
    'Brake_Percent'
  )
ORDER BY "time" ASC
"""

print("Running query...")
result = client.query(query_template)
df = result.to_pandas()
print(f"Returned {len(df)} rows")

# Check if data is empty
if df.empty:
    # Modify output path to mark the no-data case
    OUTPUT_PATH = f"./output/Run_Report_{REPORT_DATE}_NO_DATA.pdf"
    print(f"No data found for {REPORT_DATE}. Generating placeholder PDF → {OUTPUT_PATH}")

    with PdfPages(OUTPUT_PATH) as pdf:
        fig, ax = plt.subplots(figsize=(8.5, 11))
        fig.subplots_adjust(top=0.9, bottom=0.1)
        fig.suptitle(
            f"WFR25 Daily Run Report\nDate: {REPORT_DATE} (Toronto Local)",
            fontsize=12, weight="bold", y=0.97
        )

        # Add clear message to figure
        ax.axis("off")
        ax.text(
            0.5, 0.5,
            f"No data available for {REPORT_DATE}\n"
            f"UTC Range: {start_utc.isoformat()} → {end_utc.isoformat()}",
            fontsize=12,
            weight="bold",
            ha="center",
            va="center",
            color="red"
        )

        pdf.savefig(fig)
        plt.close(fig)

    print("Generated placeholder PDF indicating no data.")
    raise SystemExit(0)  # Stop the notebook safely here

# Continue only if data exists
df['time'] = pd.to_datetime(df['time'])
pivot_df = df.pivot(index='time', columns='signalName', values='sensorReading')
pivot_df = salt_dataframe(pivot_df)

# Summary Stats
summary = pivot_df.describe().loc[['min', 'mean', 'max']].T.round(2)
print("Summary statistics:")
print(summary)

Report range (Toronto local): 2025-11-08 00:00:00-05:00 → 2025-11-09 00:00:00-05:00
Converted to UTC: 2025-11-08 05:00:00+00:00 → 2025-11-09 05:00:00+00:00
Running query...
Returned 0 rows
No data found for 2025-11-08. Generating placeholder PDF → ./output/Run_Report_2025-11-08_NO_DATA.pdf
Generated placeholder PDF indicating no data.


SystemExit: 0

/home/hz/miniconda3/envs/carthings/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### PDF Report

In [24]:
# Generate PDF Report

with PdfPages(OUTPUT_PATH) as pdf:
    fig, axes = plt.subplots(4, 1, figsize=(8.5, 11))
    fig.subplots_adjust(top=0.90, bottom=0.06, hspace=0.55)

    # Title
    fig.suptitle(
        f"WFR25 Daily Run Report\n"
        f"Date: {REPORT_DATE} (Toronto Local)\n"
        f"UTC Range: {start_utc.isoformat()} → {end_utc.isoformat()}",
        fontsize=10, weight="bold", y=0.97, ha="center"
    )

    # Plot 1: DC Bus Voltage
    axes[0].plot(pivot_df.index, pivot_df["INV_DC_Bus_Voltage"], linewidth=0.9)
    axes[0].set_title("INV_DC_Bus_Voltage Over Time", pad=6)
    axes[0].set_ylabel("Voltage (V)")
    axes[0].grid(alpha=0.3)

    # Plot 2: Motor Phase A Current
    axes[1].plot(pivot_df.index, pivot_df["INV_Phase_A_Current"],
                 color="tab:orange", linewidth=0.9)
    axes[1].set_title("INV_Phase_A_Current Over Time", pad=6)
    axes[1].set_ylabel("Current (A)")
    axes[1].grid(alpha=0.3)

    # Plot 3: Brake Percent
    axes[2].plot(pivot_df.index, pivot_df["Brake_Percent"],
                 color="tab:green", linewidth=0.9)
    axes[2].set_title("Brake_Percent Over Time", pad=6)
    axes[2].set_ylabel("Brake (%)")
    axes[2].grid(alpha=0.3)

    # Plot 4: MD Summary Statistics
    axes[3].axis("off")
    axes[3].set_title("Summary Statistics (Min / Mean / Max)", fontsize=12, pad=10)

    # Convert summary to aligned markdown table using tabulate
    markdown_table = tabulate(
        summary.reset_index(),
        headers=["signalName"] + list(summary.columns),
        tablefmt="github",
        floatfmt=".2f"
    )

    # Draw monospaced text box
    axes[3].text(
        0.01, 0.98,
        markdown_table,
        va="top", ha="left",
        family="monospace",
        fontsize=8,
        linespacing=1.25,
        transform=axes[3].transAxes
    )

    pdf.savefig(fig)
    plt.close(fig)

print(f"PDF report generated successfully: {OUTPUT_PATH}")


PDF report generated successfully: ./output/Run_Report_2025-06-19.pdf
